****                        Video Game sales Analysis with different ML and Ensemble Modeling****


Import different python modules

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
df=pd.read_csv("../input/vgsales.csv")

In [ ]:
df.head(3)

In [ ]:
df.isnull().sum()

**Check null value and fillup missing values**

In [ ]:
df=df.fillna(0)

**EDA and DATA Analysis with different plotting**

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(df['Genre'])

In [ ]:
plt.figure(figsize=(15,5))
sns.countplot(df['Platform'])

In [ ]:
platGenre = pd.crosstab(df.Platform,df.Genre)
platGenre.head(5)

In [ ]:
platGenreTotal =platGenre.sum(axis=1).sort_values(ascending = False)
plt.figure(figsize=(10,15))
sns.barplot(x=platGenreTotal.values,y=platGenreTotal.index)

So You can see DS and PS2 have the most games in their platform and now, we can see the detail of genre of game in platform that have more than 1000 games. In the seaborn, it is easy to use heatrmap rather than the stacked bar, so we can use the heatmap to have a try.

In [ ]:
df.head(3)

In [ ]:
pub=df.groupby('Publisher')['Publisher'].count().sort_values(ascending = False).head(15)
sns.barplot(x=pub.values,y=pub.index)

In [ ]:
yr=df.groupby('Year')['Year'].count().sort_values(ascending = False).head(15)
plt.plot(yr)

In [ ]:
plt.figure(figsize=(10,5))
sns.pointplot(x=yr.index ,y=yr.values)

So it is showing most important year is 2009 based on others year

convert Categorical variable to numerical Variables

In [ ]:
from sklearn import model_selection, preprocessing
for c in df.columns:
    if df[c].dtype == 'object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(df[c].values)) 
        df[c] = lbl.transform(list(df[c].values))
        #x_train.drop(c,axis=1,inplace=True)

In [ ]:
df.head(3)

In [ ]:
corr=df.corr()
corr = (corr)
plt.figure(figsize=(14,14))
sns.heatmap(corr, cbar = True,  square = True, annot=True, fmt= '.2f',annot_kws=

{'size': 15},
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values)
sns.plt.title('Heatmap of Correlation Matrix')

so all sales are highly correlated with global sales which is very true..Rank is not showing highly correlated with global sales as this is shown in Matrix formation.

In [ ]:
plt.figure(figsize=(10,5))
sns.regplot(x='Rank',y='Global_Sales',data=df)

In [ ]:
plt.figure(figsize=(10,5))
sns.regplot(x='NA_Sales',y='Global_Sales',data=df)

In [ ]:
df.head(3)

In [ ]:
df.dtypes

Split test and train dataset randomly

In [ ]:
#Train-Test split
from sklearn.model_selection import train_test_split
label = df.pop('Global_Sales')
data_train, data_test, label_train, label_test = train_test_split(df, label, test_size = 0.2, random_state = 200)

In [ ]:
data_train.shape,data_test.shape

**Start with Extream gradient boosting**

In [ ]:
import xgboost as xgb
from sklearn.model_selection import KFold, train_test_split, GridSearchCV

In [ ]:
xgb_params = {
    'eta': 0.05,
    'max_depth': 5,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 1
}

In [ ]:
dtrain = xgb.DMatrix(data_train, label_train)

In [ ]:
cv_output = xgb.cv(xgb_params, dtrain, num_boost_round=1000, early_stopping_rounds=20,
    verbose_eval=50, show_stdv=False)

In [ ]:
num_boost_rounds = len(cv_output)
model = xgb.train(dict(xgb_params, silent=0), dtrain, num_boost_round= num_boost_rounds)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 13))
xgb.plot_importance(model, max_num_features=50, height=0.5, ax=ax)

**Company overall Rank is very important .This has been shown here.Most of the trees gives vote to this attribute.**

In [ ]:
dtest=xgb.DMatrix(data_test)

In [ ]:
y_predict = model.predict(dtest)
out = pd.DataFrame({'Actual_Global_Sales': label_test, 'predict_Global_Sales': y_predict,'Diff' :(label_test-y_predict)})
out[['Actual_Global_Sales','predict_Global_Sales','Diff']].head(5)

XGBOOST output seems very good with 97% accuracy

In [ ]:
sns.regplot(out['predict_Global_Sales'],out['Diff'])

In [ ]:
sns.regplot(out['Actual_Global_Sales'],out['Diff'])

In [ ]:
data_train.head(3)

As we already saw that all sales data are highly attached with global sales .Then consider only sales data to predict Global sales.

In [ ]:
lr_data_train=data_train[['NA_Sales','EU_Sales','JP_Sales','Other_Sales']]
lr_data_test=data_test[['NA_Sales','EU_Sales','JP_Sales','Other_Sales']]
lr_label_train=label_train
lr_label_test=label_test

In [ ]:
#Linear Regression
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(lr_data_train, lr_label_train)
lr_score_train = lr.score(lr_data_train, lr_label_train)
print("Training score: ",lr_score_train)
lr_score_test = lr.score(lr_data_test, lr_label_test)
print("Testing score: ",lr_score_test)

WOW accuracy improved to 99% !!!!!!!!!!!!

In [ ]:
y_pre = lr.predict(lr_data_test)

In [ ]:
out_lr = pd.DataFrame({'Actual_Global_Sales': lr_label_test, 'Predict_Global_Sales': y_pre,'Diff' :(lr_label_test-y_pre)})
out_lr[['Actual_Global_Sales','Predict_Global_Sales','Diff']].head(5)

In [ ]:
out_lr.shape

In [ ]:
sns.regplot(out_lr['Predict_Global_Sales'],out_lr['Diff'])

Lets consider Ensemble modeling ..........
**XGBOOST + Linear Regressor >>Lesso Regressor>Output**

In [ ]:
#Ensemble XGBOOST & LINEAR REGRESSOR for train data
en_dtest=xgb.DMatrix(data_train)
y_xgb_pred = model.predict(en_dtest)

y_lr_pred = lr.predict(lr_data_train)

Ensemble=pd.DataFrame({'XGBOOST':y_xgb_pred ,'LINEAR_REG':y_lr_pred ,'GLOBAL_SALES':lr_label_train})


In [ ]:
Ensemble.head(5)

In [ ]:
corr=Ensemble.corr()
corr = (corr)
sns.heatmap(corr, 
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values)
sns.plt.title('Heatmap of Correlation Matrix')

In [ ]:
#Ensemble XGBOOST & LINEAR REGRESSOR for test data
en_dtest_1=xgb.DMatrix(data_test)
y_xgb_pred_1 = model.predict(en_dtest_1)

y_lr_pred_1 = lr.predict(lr_data_test)

Ensemble_test_with_actual=pd.DataFrame({'XGBOOST':y_xgb_pred_1 ,'LINEAR_REG':y_lr_pred_1,'ACTUAL_SALES':lr_label_test})

In [ ]:
Ensemble_test=Ensemble_test_with_actual[['XGBOOST','LINEAR_REG']]

In [ ]:
Ensemble_test.shape,Ensemble.shape

In [ ]:
Ensemble_test.head(3)

In [ ]:
#Train-Test split
from sklearn.model_selection import train_test_split
label = Ensemble.pop('GLOBAL_SALES')
X_train, X_test, Y_train, Y_test = train_test_split(Ensemble, label, test_size = 0.3, random_state = 200)

we used lesso as 2 attribute is highly correlated...means xgboost output and linear regressor output...U can try different model also

In [ ]:
from sklearn import linear_model
clf = linear_model.Lasso(alpha=1e-4)
clf.fit(X_train,Y_train)
tr_scr=clf.score(X_train,Y_train)
print("Training score: ",tr_scr)
ts_scr=clf.score(X_test,Y_test)
print("Testing score: ",ts_scr)


In [ ]:
ensm_prd=clf.predict(Ensemble_test)

In [ ]:
lr_label_test.shape

In [ ]:
Output=pd.DataFrame({'LINEAR_REGRASSOR':Ensemble_test['LINEAR_REG'],'XGBOOST':Ensemble_test['XGBOOST'],'ENSEMBLE':ensm_prd,'ACTUAL_PRICE':Ensemble_test_with_actual['ACTUAL_SALES']})

In [ ]:
Output.head(10)